# Wild Edible Food Application with Image Recognition 

#### A CNN trained using the swedish leaf dataset to classify images from a set of 15 species

In [1]:
import numpy as np
import os
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import random
from keras.utils import to_categorical

#datasetDir = r"D:\Uni_Work\Third Year\Advanced Intelligent Systems\TensorFlow\swedishLeafDataset"
#testDir = r"D:\Uni_Work\Third Year\Advanced Intelligent Systems\TensorFlow\data\test"
#ext_test_data = r"D:\Uni_Work\Third Year\Advanced Intelligent Systems\TensorFlow\data\External_test"
cwd = os.getcwd()
datasetDir = cwd +"\data\\train"
testDir = cwd + "\data\\test"

class_names = ['leaf1', 'leaf2', 'leaf3', 'leaf4', 'leaf5',
               'leaf6', 'leaf7', 'leaf8', 'leaf9', 'leaf10',
               'leaf11', 'leaf12', 'leaf13', 'leaf14', 'leaf15']

In [2]:
training_data = []
test_data = []
IMG_SIZE = 50


def create_training_data():
    for category in class_names:
        path = os.path.join(datasetDir, category)
        class_num = class_names.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE) #
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass


def create_test_data():
    for category in class_names:
        path = os.path.join(testDir)
        class_num = class_names.index(category)
        for img in range(0, 7):
            file_name = "l" + str(class_names.index(category)+1) + "nr00" + str(img + 1) + ".TIF"
            img_array = cv2.imread(os.path.join(path, file_name), cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            test_data.append([new_array, class_num])
            
create_training_data()
create_test_data()
random.shuffle(training_data)
random.shuffle(test_data)

In [5]:
X = []
Y = []

for features, label in training_data:
    X.append(features)
    Y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
print(X.shape)


X_test = []
Y_test = []
for features, label in test_data:
    X_test.append(features)
    Y_test.append(label)
X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

X = X.astype("float32")
X_test = X_test.astype("float32")
X /= 255
X_test /= 255


Y = to_categorical(Y, 15)
Y_test = to_categorical(Y_test, 15)

(1020, 50, 50, 1)


In [6]:
ext_test_data = cwd + "\data\\External_test"
ext_test = []
def ext_create_test_data():
    for category in class_names:
        path = os.path.join(ext_test_data)
        class_num = class_names.index(category)
        for img in range(0, 2):
            file_name = "exl" + str(class_names.index(category)+1) + "nr00" + str(img + 1) + ".JPG"
            img_array = cv2.imread(os.path.join(path, file_name), cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            ext_test.append([new_array, class_num])
            
ext_create_test_data()
#random.shuffle(ext_test())
X_ext = []
Y_ext = []
for features, label in ext_test:
    X_ext.append(features)
    Y_ext.append(label)
X_ext = np.array(X_ext).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y_ext = to_categorical(Y_ext, 15)

In [7]:
import tensorflow as tf
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 1)))#, data_format="channels_first"


model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(15, activation="softmax"))


model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy", tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
model.fit(X, Y, batch_size=32, epochs=15, verbose=1)


Epoch 1/15
32/32 [==============================] - 2s 70ms/step - loss: 2.2944 - accuracy: 0.3069 - recall_1: 0.0431 - precision_1: 0.9565
Epoch 2/15
32/32 [==============================] - 2s 69ms/step - loss: 1.2133 - accuracy: 0.6167 - recall_1: 0.3902 - precision_1: 0.8172
Epoch 3/15
32/32 [==============================] - 2s 70ms/step - loss: 0.7059 - accuracy: 0.7755 - recall_1: 0.6706 - precision_1: 0.8724
Epoch 4/15
32/32 [==============================] - 2s 73ms/step - loss: 0.5306 - accuracy: 0.8353 - recall_1: 0.7706 - precision_1: 0.8983
Epoch 5/15
32/32 [==============================] - 2s 72ms/step - loss: 0.4337 - accuracy: 0.8618 - recall_1: 0.8157 - precision_1: 0.9043
Epoch 6/15
32/32 [==============================] - 2s 69ms/step - loss: 0.3464 - accuracy: 0.8941 - recall_1: 0.8490 - precision_1: 0.9342
Epoch 7/15
32/32 [==============================] - 2s 72ms/step - loss: 0.2708 - accuracy: 0.9206 - recall_1: 0.8873 - precision_1: 0.9417
Epoch 8/15
32/32 [==

In [8]:
#from sklearn import classification_report
score = model.evaluate(X_test, Y_test, verbose=0)
score2 = model.evaluate(X, Y, verbose=0)
print(score)
print(score2)
score3 = model.evaluate(X_ext, Y_ext, verbose=0)
print(score3)

#print(classification_report(Y_ext, model.predict(X_ext)))
print(model.metrics_names)

[1.970278024673462, 0.8095238208770752, 0.8095238208770752, 0.8095238208770752]
[0.011373880319297314, 0.9990196228027344, 0.9970588088035583, 0.9990176558494568]
[2864.52197265625, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224]
['loss', 'accuracy', 'recall_1', 'precision_1']


In [10]:
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mping
#imgPath = "data/Percentage_Image/exl11nr002-Copy1.jpg"
imgPath = "data/Percentage_Image/predict2.tif"

#img = image.load_img(imgPath, target_size=(50, 50))

#img_array = image.img_to_array(img)
#img_batch = np.expand_dims(img_array, axis=0)

#img_preprocessed = preprocess_input(img_batch)
#print(img_preprocessed.shape)
#prediction = model.predict(img_preprocessed)

#img = image.load_img(imgPath, target_size=(50, 50))
#plt.imshow(img)
#plt.show()


def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

prediction = model.predict([prepare(imgPath)])
print(prediction)


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


In [11]:
prediction = model.predict([prepare(imgPath)])
print(prediction)
print(class_names[int(prediction[0][0])])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
leaf1


In [12]:
prediction = model.predict([prepare("data/Percentage_Image/predict1.jpg")])
print(prediction)
print(class_names[int(prediction[0][0])])

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
leaf1


In [13]:
prediction = model.predict([prepare("data/train/leaf7/L7nr067.tif")])
print(prediction)
print(class_names[int(prediction[0][0])])

[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
leaf1


In [14]:
prediction = model.predict_classes([prepare("data/train/leaf2/L2nr067.tif")])
print(class_names[int(prediction)])

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
leaf2
